# Data cleaning 

In [1]:
import pandas as pd

In [2]:
from use_cases.contributions import create_table_contributions

from use_cases.emotions import create_table_emotions
from use_cases.personal_needs import create_table_personal_needs
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading Data Surveys

In [3]:
chunksize   = 100 # Just for debugging
survey_path = './data/BBDD_Dialogos_2021_01_27.csv' 
survey = pd.read_csv(survey_path, chunksize=chunksize, low_memory=False)
for survey in survey: break
    
ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

In [4]:
survey.sample(2)
ind_online_survey.sample(2)

,Submission Date,RUN,Edad,Comuna,1. ¿Cuál es el nivel de educación alcanzado por Usted?,2. ¿Cuál es el nivel de educación que alcanzó el jefe de su hogar? (sólo si se trata de alguien distinto a Usted),1 >> Emociones / Sentimientos / Sensaciones,1 >> Explique lo mencionado,2 >> Emociones / Sentimientos / Sensaciones,2 >> Explique lo mencionado,...,2 >> Rol del actor social (Describa),3 >> Necesidades del país identificadas,3 >> Rol del Estado (Describa),"3 >> Actor social (empresa, organizaciones sociales, medios de comunicación, comunidad, etc)",3 >> Rol del actor social (Describa),5. Pregunta: ¿Cuál es mi contribución personal para construir el Chile que queremos?,Comuna.1,"Comentarios finales: Si tiene alguna observación o sugerencia, en este espacio puede indicarla.",Región,Región.1
1411,2019/12/09 16:08:10,648b91bb0ad8e4156479a09900ff212a,28,La calera,Universitaria completa.,Media completa. Técnica incompleta.,Preocupación,No saber si el gobierno va a hacer lo correcto...,Estres,me gebera agotamiento la situación donde no pu...,...,NaN,Salud,Aumentar la inversión a la red publica de aten...,El gobierno es el único responsable,NaN,Mi contribución personal va enfocada a partici...,NaN,El gobierno sabe cuales son los cambios que de...,NaN,Valparaíso
790,2019/12/31 09:53:12,f08bbda9a1c8c403304b1675d5285852,28,La Florida,Universitaria completa.,Universitaria incompleta. Técnica completa.,Angustia,Mucha angustia al día siguiente de las quemas ...,Rabia,"Contra las élites y el gobierno, es incomprens...",...,El estado falla en esto,Desigualdad Horizontal,Garantizar igualdad de acceso a infraestructur...,ninguno,El estado falla en esto,"Participar en la dirigencia de mi comunidad, a...",NaN,"Es necesario:\n1. Regular a las AFP, poner alt...",Región Metropolitana de Santiago,NaN


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [323]:
persons_table = create_table_persons(survey)

In [324]:
persons_table.sample(3)

,id,diag_id,age,sex,level,comuna_id,age_range
43,143,enc_u_4598371090155421436,26,f,universitaria_completa,7101,15-30
39,139,enc_u_4598379130154541284,36,h,basica_completa,7101,30-45
71,71,enc_u_4597419179157584588,62,f,media_incompleta,9105,>=60


### Dialogues

In [360]:
dialogues_table = create_table_dialogues(survey)

In [356]:
dialogues_table.sample(3)

,id,date,init_time,end_time,location,address,comuna_id,n_members,group_name
71,enc_u_4597419179157584588,06-03-2020,11:00:00,12:40:00,centro cultural freire,canales sur #186,9105,5,usuarios de oficina movil de bienes nacionales
79,enc_u_4596806191197827612,22-12-2019,20:30:00,21:30:00,eiffel 582,eiffel 582 villa pedro lagos,15101,12,usuarios de oficina movil de bienes nacionales
97,enc_u_4595907200156285158,31-01-2020,22:10:00,23:50:00,brisas de san miguel,,7101,5,usuarios de oficina movil de bienes nacionales


### Emotions

<img src="diagrams/q1.png" width=730 height=520 />

In [380]:
%%time
emotions_table = create_table_emotions(survey,ind_survey_path, ind_online_survey)

CPU times: user 38.9 s, sys: 2.01 s, total: 40.9 s
Wall time: 41 s


In [547]:
emotions_table.sample(2)

,source_id,name,name_tokens,macro,exp,exp_tokens,is_dialogue,is_online,id
59,ENC_U_4597643380118026494,pena,[pena],pena,- al ver como se pierde y no se valora la inst...,"[ver, pierde, valora, institucionalidad, gente...",1,0,255
9737,4839317,tristeza,[tristeza],,por los destrozos que han hecho a lo largo de ...,"[destrozos, hecho, largo, pais, perjudicando, ...",0,0,9537


### Individuals

In [11]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey)

CPU times: user 7.37 s, sys: 28.8 ms, total: 7.4 s
Wall time: 7.35 s


In [12]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,online,age_range
2059,3094162,15-01-2020,65,4301,Educación básica incompleta o inferior,,>=60
6777,3449396,27-01-2020,59,6301,Media incompleta (incluyendo Media Técnica),,45-60


### Country Needs

In [307]:
%%time
country_needs = create_table_country_needs(survey, ind_survey, ind_survey_path, ind_online_survey_path)

CPU times: user 1min 14s, sys: 2.97 s, total: 1min 17s
Wall time: 1min 17s


In [317]:
country_needs.sample(3)

,source_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_dialogue,is_online,id
16703,5129661,desigualdad,[desigualdad],,,,,,,,0,0,16487
2167,3400383,equidad,[equidad],,,,,,estado,1,0,0,21770
260,2997498,salud,[salud],,,,el ministro que se la juegue,"[ministro, juegue]",estado,,0,0,962


### Family/PersonalNeeds

In [305]:
%%time
personal_needs = create_table_personal_needs(survey, ind_survey, ind_online_survey)

CPU times: user 56.2 s, sys: 3.65 s, total: 59.9 s
Wall time: 59.9 s


In [306]:
personal_needs.sample(3)

,source_id,name,name_tokens,exp,exp_tokens,macro,priority,is_dialogue,is_online,id
11961,5838240,inseguridad,[inseguridad],el no vivir en forma tranquila por la delincue...,"[vivir, forma, tranquila, delincuencia, debier...",inseguridad,3.0,0,0,36931
6881,3378109,"trabajo, tranquilidad","[trabajo, tranquilidad]","para ir a mi pega, que no hayan incidentes","[ir, pega, incidentes]","trabajo, tranquilidad",1.0,0,0,6239
52,3d897e8dea843aaaf7a8fb97983274f4,economica,[economica],"aumentar las pensiones, terminar con las afp, ...","[aumentar, pensiones, terminar, afp, crear, nu...",economica,1.0,0,1,11766


### Contributions

In [303]:
contributions = create_table_contributions(survey, ind_survey, ind_online_survey)

In [304]:
contributions.sample(3)

,id,source_id,text,tokens,macro,is_dialogue,is_online
4816,5316,3203816,ninguna,[ninguna],ninguna,0,0
8492,8992,4196077,una buena reinsercion laboral y ser un aporte ...,"[buena, reinsercion, laboral, ser, aporte, soc...",una buena reinsercion laboral y ser un aporte ...,0,0
3506,4006,3159647,cada dia ser mejor persona,"[cada, dia, ser, mejor, persona]",cada dia ser mejor persona,0,0
